In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Load datasets
customers = pd.read_csv("data/Customers.csv")
products = pd.read_csv("data/Products.csv")
transactions = pd.read_csv("data/Transactions.csv")

In [4]:
# Merge datasets
data = transactions.merge(customers, on="CustomerID").merge(products, on="ProductID")
data = data.drop(columns=["Price_y"]).rename(columns={"Price_x": "Price"})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price            1000 non-null   float64
 7   CustomerName     1000 non-null   object 
 8   Region           1000 non-null   object 
 9   SignupDate       1000 non-null   object 
 10  ProductName      1000 non-null   object 
 11  Category         1000 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 93.9+ KB


In [5]:
# Encode categorical features
encoder = LabelEncoder()
data["Region"] = encoder.fit_transform(data["Region"])
data["Category"] = encoder.fit_transform(data["Category"])

In [6]:
# Aggregate transaction data per customer
customer_features = data.groupby("CustomerID").agg({
    "TotalValue": "sum",
    "Quantity": "sum",
    "Region": "first",
    "Category": lambda x: x.mode()[0]  # Most purchased category
}).reset_index()

In [8]:
# Normalize numerical features
customer_features[["TotalValue", "Quantity"]] = customer_features[["TotalValue", "Quantity"]].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Compute similarity matrix
feature_matrix = customer_features.drop(columns=["CustomerID"])
similarity_matrix = cosine_similarity(feature_matrix)

# Generate Lookalike recommendations
customer_ids = customer_features["CustomerID"].tolist()
lookalikes = {}

for i, cust_id in enumerate(customer_ids[:20]):  # First 20 customers
    similar_indices = np.argsort(similarity_matrix[i])[::-1][1:4]  # Top 3 similar customers
    similar_customers = [(customer_ids[idx], similarity_matrix[i][idx]) for idx in similar_indices]
    lookalikes[cust_id] = similar_customers

# Save as CSV
# lookalike_df = pd.DataFrame.from_dict(lookalikes, orient='index')
# lookalike_df.to_csv("Lookalike.csv", header=False)


# Format the lookalikes data in the required format
lookalike_map = []
for key, value in lookalikes.items():
    lookalike_list = [f"('{lookalike_id}', {similarity:.6f})" for lookalike_id, similarity in value]
    lookalike_map.append([key, f"[{', '.join(lookalike_list)}]"])

lookalike_df = pd.DataFrame(lookalike_map, columns=['CustomerID', 'Lookalikes'])

# Save as CSV
lookalike_df.to_csv("Raja_Majumdar_Lookalike.csv", index=False)